# Assignment2: Logistic Regression 구현
1) 제가 제시한 틀을 사용하여 구현을 할 필요는 없습니다. 원하시는 대로 구현하시면 됩니다.  
2) 단, code copy는 허용하지 않습니다. 코드 주석을 꼭 상세히 달아주세요. 주석이 부족한 경우, 미제출로 간주합니다.

In [1]:
import math, random
from functools import partial,reduce
from assignment2 import *
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
"""
data 설명
1) Label: 유료 계정 등록 여부(target)
2) bias: 회귀 모형에서의 상수항을 위한 term (추정 시 포함하지 않아도 ok)
3) experience: 근속연수
4) salary: 연봉

어떤 사용자가 유료 계정을 등록할지(Label == 1)에 대한 예측을 로지스틱 회귀 모형으로 진행합니다.
"""

'\ndata 설명\n1) Label: 유료 계정 등록 여부(target)\n2) bias: 회귀 모형에서의 상수항을 위한 term (추정 시 포함하지 않아도 ok)\n3) experience: 근속연수\n4) salary: 연봉\n\n어떤 사용자가 유료 계정을 등록할지(Label == 1)에 대한 예측을 로지스틱 회귀 모형으로 진행합니다.\n'

In [3]:
data = pd.read_csv('assignment_2.csv')

In [4]:
data

,Label,bias,experience,salary
0,1,1,0.7,48000
1,0,1,1.9,48000
2,1,1,2.5,60000
3,0,1,4.2,63000
4,0,1,6.0,76000
5,0,1,6.5,69000
6,0,1,7.5,76000
7,0,1,8.1,88000
8,1,1,8.7,83000
9,1,1,10.0,83000


### 0. Gradient Descent

In [5]:
def step(v, direction, step_size):
    """
    한 지점에서 step size만큼 이동하는 step 함수를 구현하세요.
    필요한 인자는 3가지입니다.
    """
    return [v_i + step_size*direction_i for v_i, direction_i in zip(v,direction)]

In [6]:
def safe(f):
    """
    f에 대한 예외처리를 위한 함수(f가 infinite일 때)
    """
    def safe_f(*args, **kargs):
        try:
            return f(*args, **kwargs)
        except:
            return float('inf')
    return safe_f

In [7]:
def minimize_bgd(target_fn, gradient_fn, theta_0, tolerance = 0.00001): # bgd: batch gradient descent
    """
    목적함수를 최소화시키는 theta를 경사 하강법을 사용해서 찾는다.
    """
    step_sizes = [100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
    
    # 시작점 설정
    theta = theta_0
    target_fn = safe(target_fn) # 오류를 처리할 수 있는 target_fn으로 변환
    value = target_fn(theta) # 최소화시키려는 값
    
    while True:
        gradient = gradient_fn(theta) # gradient값 계산
        next_thetas = [step(theta, gradient, -step_size) for step_size in step_sizes] #### update thetas --> 각 step sizes에 따른 theta값을 list형태로 리턴
        
        # 함수를 최소화시키는 theta 선택
        obj =  next_thetas ## 알맞은 obj, key 값을 채워넣으세요.
        key = target_fn
        next_theta = min(obj, key = key)
        next_value = target_fn(next_theta)
        
        # tolerance만큼 수렴하면 멈춤
        temp = abs(value-next_value) # temp 채워넣으세요
        if temp < tolerance:
            return theta
        else: #### 어떻게 업데이트 시킬지 채워넣으세요
            theta, value = next_theta, next_value
        

In [8]:
def stochastic():
    """
    sgd 구현 (추가적인 부분이니 필수는 아닙니다.)
    """
    # random sampling 파트
    

### 1. 로지스틱 함수
해당 함수는 1/(1+exp[-(ax+b)]로 표현되었음을 기억합시다.

In [9]:
def logistic(x):
    try:
        return 1.0 / (1 + math.exp(-x))
    except OverflowError:
        return 1e-9

In [10]:
# 추가문제: softmax 구현
def softmax(v): # 벡터를 인자로 받는다
    summation = sum([math.exp(v_i) for v_i in arr])
    return [math.exp(v_i)/summation for v_i in v]

In [11]:
arr = [0.3, 2.9, 4.0]
softmax(arr)

[0.018211273295547534, 0.2451918129350739, 0.7365969137693785]

### 2. 목적함수 구현
최적화할 목적함수를 구현합니다.

In [12]:
def logistic_log_likelihood_i(x_i, y_i, beta): # 개별 데이터포인트에 대한 likelihood 값
    """
    None 부분을 채워서 함수를 완성하세요.
    """
    cond = y_i == 1
    if cond:
        return math.log(logistic(dot(x_i, beta)))
    else:
        return math.log(1-logistic(dot(x_i, beta)))

In [13]:
def logistic_log_likelihood(x,y,beta): # 전체 데이터에 대한 likelihood
    """
    함수의 인자를 채워넣고,return 문을 완성하세요.
    """
    return sum(logistic_log_likelihood_i(x_i,y_i,beta) for x_i,y_i in zip(x,y))

### 3. Gradient for Log Reg
아래 3가지 함수에 대해 해당 함수의 인자와 기능을 자세히 설명하세요.

In [14]:
def logistic_log_partial_ij(x_i, y_i, beta, j):
    """
    i번째 데이터에 대해서 j번째 열"""
    return (y_i - logistic(dot(x_i, beta))) * x_i[j] # j: beta의 index

def logistic_log_gradient_i(x_i, y_i, beta):
    return [logistic_log_partial_ij(x_i, y_i, beta, j) for j, _ in enumerate(beta)]

def logistic_log_gradient(x, y, beta):
    return reduce(vector_add, [logistic_log_gradient_i(x_i, y_i, beta) for x_i, y_i in zip(x,y)])

###설명###

### 4. Model Fitting
위에서 구현한 목적함수와 Gradient Descent Algorithm을 이용하여 Model을 Fitting 시켜보세요.  

In [15]:
X = data.drop('Label', axis = 1)
y = data['Label']

In [16]:
X=X.as_matrix()
y=y.as_matrix()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [17]:
random.seed(0)
x_train, x_test, y_train, y_test = train_test_split(X, y)

In [ ]:
# 아래에 Model Fitting 진행
# partial을 이용해 fn과 gradient_fn 구현

fn = partial(logistic_log_likelihood, x_train, y_train)
gradient_fn = partial(logistic_log_gradient, x_train, y_train)

beta_0 = [random.random() for _ in range(3)] # 임의의 시작점

# 경사 하강법으로 최적화
beta_hat = minimize_bgd(neg(fn), neg_all(gradient_fn), beta_0)